In [1]:
try:
    sc.stop()
except:
    pass


In [2]:
import re 

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType


conf = SparkConf().setAppName('Spark-logAnalysis').setMaster('local[4]')
sc = SparkContext.getOrCreate(conf = conf)


In [3]:

spark = SparkSession.builder.appName('sparklog').getOrCreate()



In [4]:
spark

In [10]:
rdd = sc.textFile("file:///home/hadoop/data/access_log_Jul95")

In [11]:
rdd.count()

1891715

In [12]:
rdd.take(5)


['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085',
 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0',
 '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179']

In [13]:
#expr=r'^(\S+)((\s)(-))+\s(\[\S+ -\d{4}\])\s("\w+\s+([^\s]+)\s+HTTP.*")\s(\d{3}\s(\d*)$)'
expr=r"((\d+.\d+.\d+.\d+)|[\w\.]*)\s[-]\s[-]\s[[](\d{2}[/]\w{3}[/]\d{4}[:]\d{2}[:]\d{2}[:]\d{2})\s([-]\d{4})[]]\s([\"].*[\"])\s(\d*)\s([\-]|\d*|)[\n]*"


def log_expression(line):
    # Tried to include match object but got "TypeError: can't pickle _sre.SRE_Match objects" inside  result_rdd.take(1)
    match = re.search(expr,line)
    if match is None:    
        return(line,0)
    else:
        return(line,1)


def match_log(line):
    match = re.search(expr,line)
    return(match)

In [ ]:
# x = match_log(rdd.take(5)[4])

#dir(x)
#display(x.groups())

keys_dict = {0: 'hostname',
             2: 'datetime_s',
             3: 'tz',
             4: 'request',
             5: 'response',
             6: 'size'}

# [(it[0], it[1], x.groups()[it[0]]) for it in keys_dict.items()]

In [ ]:
# n_logs = rdd.count()
# failed = rdd.map(lambda line: log_expression(line)).filter(lambda line: line[1] == 0).count()
# print('Out of a total of {} logs, {} failed to parse'.format(n_logs,failed))
# # Get the failed records line[1] == 0
# failed1=rdd.map(lambda line: log_expression(line)).filter(lambda line: line[1]==0)
# print("failed lines:")
# print(failed1.take(3))
# print("*"*20)

# success = rdd.map(lambda line: log_expression(line).filter(lambda line: line[1]==1))
# n_logs = rdd.count()

In [ ]:
n_logs = rdd.count()

parsed = rdd.map(lambda line: log_expression(line))
failed = parsed.filter(lambda line: line[1] == 0)
success = parsed.filter(lambda line: line[1] == 1)
del parsed
print('Out of a total of {} logs, {} failed to parse'.format(n_logs,failed.count()))
print("failed lines:")
print(failed.take(3))


In [ ]:
display(type(success),
dir(success))

In [ ]:
success.take(1)
# hostname

In [ ]:
# #https://sparkbyexamples.com/pyspark/pyspark-create-an-empty-dataframe/
# schema = StructType([
#     StructField('hostname', StringType(), True), 
#     StructField('datetime_s', StringType(), True), 
#     StructField('tz', StringType(), True), 
#     StructField('request', StringType(), True), 
#     StructField('response', StringType(), True), 
#     StructField('size', StringType(), True)])

# # emptyrdd= sc.parallelize([])

# # df = spark.createDataFrame(emptyrdd, schema)
# # df = sc.parallelize([]).toDF(schema)
# # df = spark.createDataFrame([],schema)

# df = spark.createDataFrame([('Tom', 80), (None, 60), ('Alice', None)], ["name", "height"])

In [ ]:
print(keys_dict)
#date format #https://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html

In [ ]:
empty = SQLContext.createDataFrame(sc.emptyRDD(), schema)

In [ ]:
from pyspark.sql.types import DataType

In [ ]:
issubclass(StringType, DataType), isinstance(StringType, DataType)

In [ ]:
isinstance(DataType, DataType)

In [15]:

logs_df = spark.read.text('file:///home/hadoop/data/access_log_Jul95')


In [38]:
logfile_df = logs_df
sample_logs = logs_df.sample(withReplacement=None, fraction=.01, seed=0)

In [36]:
sample_logs.take(5)
sample_logs

AttributeError: 'DataFrame' object has no attribute 'Rows'

In [27]:

import re 
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'

hosts = [re.search(host_pattern, item).group(1)
        if re.search(host_pattern, item)
        else 'no match'
        for item in sample_logs]

ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
timestamps = [re.search(ts_pattern, item).group(1) for item in sample_logs]
timestamps

method_uri_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
method_uri_protocol = [re.search(method_uri_pattern, item).groups() 
                       if re.search(method_uri_pattern, item)
                       else 'no match'
                       for item in sample_logs]
method_uri_protocol



from pyspark.sql.functions import regexp_extract

logfile_df = logs_df.select(regexp_extract('value', host_pattern, 1).alias('host'), 
                     regexp_extract('value', ts_pattern, 1).alias('timestamp'),
                     regexp_extract('value', method_uri_pattern, 1).alias('method'),
                     regexp_extract('value', method_uri_pattern, 2).alias('endpoint'),
                     regexp_extract('value', method_uri_pattern, 3).alias('protocol'), 
                     regexp_extract('value', status_pattern, 1).alias('status'), 
                     regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))
logfile_df.show(10)

TypeError: expected string or bytes-like object

In [35]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum

def count_na(col_name):
    return spark_sum(col (col_name).isNull().cast('integer'))

In [39]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum

def count_na(col_name):
    return spark_sum(col(col_name).isNull().cast('integer')).alias(col_name)

# Build a list of column expressions , one per column
exprs = [count_na(col_name) for col_name in logfile_df.columns]

# *exprs converts the list of expression into variable function arguments i.e. unpacks the list
logfile_df.agg(*exprs).show()


+-----+
|value|
+-----+
|    0|
+-----+



In [41]:
logfile_df.printSchema()

root
 |-- value: string (nullable = true)



In [ ]:
# RDD - persistance & cache
# cache - IN Memory
# pesist - DISK_ONLY, MEMORY_AND_DISK, MEMORY_ONLY
logfile_df.cache()

#### EDA & Statistics

In [42]:
logfile_df.describe(['content_size'])

AnalysisException: "cannot resolve '`content_size`' given input columns: [value];;\n'Project ['content_size]\n+- Relation[value#0] text\n"

In [44]:
logfile_df.describe('value')

DataFrame[summary: string, value: string]

In [ ]:
# logfile_df.describe(['content_size']).toPandas()
logfile_df.describe(['content_size']).toPandas()

In [ ]:
status_code_df = (logfile_df.groupby('status').count(),sort('status').cache())

In [ ]:
status_code_df.count()

In [ ]:
status_code_df.toPandas().sort_values(by=['count'], ascending=False)

In [ ]:
status_code_final = status_code_df.toPandas().sort_values(by = ['count'], ascending=False)
status_code_final


In [45]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

ImportError: No module named 'seaborn'

In [ ]:
sns.catplot(x='status', y='count', data=status_code_final, kind='bar')

In [ ]:
from pyspark.sql.functions import log

log_freq_df = status_code_df.withColumn('log(count)', log(status_code_df['count']))

In [46]:
from pyspark.sql.functions import log
help(log)

Help on function log in module pyspark.sql.functions:

log(arg1, arg2=None)
    Returns the first argument-based logarithm of the second argument.
    
    If there is only one argument, then this takes the natural logarithm of the argument.
    
    >>> df.select(log(10.0, df.age).alias('ten')).rdd.map(lambda l: str(l.ten)[:7]).collect()
    ['0.30102', '0.69897']
    
    >>> df.select(log(df.age).alias('e')).rdd.map(lambda l: str(l.e)[:7]).collect()
    ['0.69314', '1.60943']
    
    .. versionadded:: 1.5



In [ ]:
log_freq_final = log_freq_df.toPandas().sort_values(by=['log(count)'], ascending=False)
